In [1]:
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
token="здесь должен быть токен"

In [3]:
group_name="expert_news"
offset=0
count=100
all_posts=[]
all_comments=[]

In [4]:
while offset<1000:
    req=requests.get('https://api.vk.com/method/wall.search',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'domain': group_name,
                   'query': 'инфляция',
                   'count': count,
                   'offset': offset
                   
               })
    res=req.json()['response']['items']
    offset+=100
    all_posts.extend(res)

In [5]:
df = pd.json_normalize(all_posts)
df

,id,from_id,owner_id,date,marked_as_ads,short_text_rate,attachments,is_favorite,post_id,parents_stack,...,likes.user_likes,likes.can_publish,post_source.type,reposts.count,reposts.user_reposted,zoom_text,views.count,post_source.platform,edited,carousel_offset
0,261147,21199039,-26517977,1674567497,0,0.8,[],False,261147.0,[261157],...,0,1,vk,0,0,NaN,NaN,NaN,NaN,NaN
1,261147,254867,-26517977,1674490154,0,0.8,[],False,261147.0,[],...,0,1,vk,0,0,True,NaN,NaN,NaN,NaN
2,260957,-26517977,-26517977,1673608200,0,0.8,"[{'type': 'link', 'link': {'url': 'https://exp...",False,NaN,NaN,...,0,1,vk,5,0,NaN,1311.0,NaN,NaN,NaN
3,260939,-26517977,-26517977,1673521800,0,0.8,"[{'type': 'link', 'link': {'url': 'https://exp...",False,NaN,NaN,...,0,1,api,0,0,NaN,820.0,iphone,NaN,NaN
4,260352,-26517977,-26517977,1671609989,0,0.8,"[{'type': 'link', 'link': {'url': 'https://exp...",False,NaN,NaN,...,0,1,api,1,0,NaN,762.0,iphone,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,817,-26517977,-26517977,1308052912,0,0.8,"[{'type': 'link', 'link': {'url': 'http://expe...",False,NaN,NaN,...,0,1,vk,1,0,NaN,NaN,NaN,NaN,NaN
985,786,-26517977,-26517977,1307363844,0,0.8,"[{'type': 'link', 'link': {'url': 'http://expe...",False,NaN,NaN,...,0,1,vk,1,0,NaN,NaN,NaN,NaN,NaN
986,548,-26517977,-26517977,1306001444,0,0.8,"[{'type': 'link', 'link': {'url': 'http://expe...",False,NaN,NaN,...,0,1,vk,1,0,NaN,NaN,NaN,NaN,NaN
987,394,-26517977,-26517977,1305557129,0,0.8,"[{'type': 'link', 'link': {'url': 'http://expe...",False,NaN,NaN,...,0,1,vk,0,0,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     989 non-null    int64  
 1   from_id                989 non-null    int64  
 2   owner_id               989 non-null    int64  
 3   date                   989 non-null    int64  
 4   marked_as_ads          989 non-null    int64  
 5   short_text_rate        989 non-null    float64
 6   attachments            989 non-null    object 
 7   is_favorite            989 non-null    bool   
 8   post_id                602 non-null    float64
 9   parents_stack          602 non-null    object 
 10  post_type              989 non-null    object 
 11  text                   989 non-null    object 
 12  hash                   989 non-null    object 
 13  donut.is_donut         989 non-null    bool   
 14  comments.can_post      989 non-null    int64  
 15  commen

In [7]:
df=df[["id", "owner_id", "date", "text", "comments.count"]]
df.rename(columns={'date': 'post_time','text':'post_text'}, inplace=True)
df=df[df['post_time']>=1630454400]
df=df[df['comments.count']!=0]

In [8]:
owner_id=-26517977
posts_id=list(df['id'])

In [9]:
for post_id in posts_id:
    req=requests.get('https://api.vk.com/method/wall.getComments',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'owner_id': owner_id,
                   'post_id': post_id
               })
    time.sleep(1)
    res=req.json()['response']['items']
    all_comments.extend(res)

In [10]:
data = pd.json_normalize(all_comments)
data

,id,from_id,date,text,post_id,owner_id,parents_stack,attachments,thread.count,thread.items,thread.can_post,thread.show_reply_button,deleted
0,261149,676700549,1674461640,А НА СКОЛЬКО % ПРОЦЕНТОВ СНИЗИЛА ВЫСОКАЯ СТАВК...,261147.0,-26517977.0,[],"[{'type': 'photo', 'photo': {'album_id': 28436...",0,[],True,True,NaN
1,261153,710313517,1674481392,"Если бы не падение рубля с 52 до 72, доходност...",261147.0,-26517977.0,[],NaN,0,[],True,True,NaN
2,261157,254867,1674490154,Это ниже инфляции. Они потеряли деньги по сути.,261147.0,-26517977.0,[],NaN,1,[],True,True,NaN
3,260959,709848164,1673608933,"Уси-пуси, я обиделась...🤣",260957.0,-26517977.0,[],NaN,0,[],True,True,NaN
4,260960,147845069,1673610014,Есть же и в Евросоюзе вменяемые главы гос-в...,260957.0,-26517977.0,[],NaN,0,[],True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,244873,418165222,1630488189,Ведь почти во всех регионах цены в самом деле ...,244870.0,-26517977.0,[],NaN,0,[],True,True,NaN
191,244874,315402790,1630488801,"Согласно статистике, в Алтае, Забайкалье, Калу...",244870.0,-26517977.0,[],NaN,0,[],True,True,NaN
192,244875,640119947,1630489632,"Рост цен на продукты неизбежен, но Путин старе...",244870.0,-26517977.0,[],NaN,0,[],True,True,NaN
193,244888,1693057,1630508529,Национальное достояние уходит не в те руки,244870.0,-26517977.0,[],NaN,0,[],True,True,NaN


In [11]:
data=data[["post_id", "text", "date"]]
data.rename(columns={'date': 'comment_time', 'id': 'comment_id'}, inplace=True)
data=data.dropna()
data['post_id'] = data['post_id'].astype(int)
data

,post_id,text,comment_time
0,261147,А НА СКОЛЬКО % ПРОЦЕНТОВ СНИЗИЛА ВЫСОКАЯ СТАВК...,1674461640
1,261147,"Если бы не падение рубля с 52 до 72, доходност...",1674481392
2,261147,Это ниже инфляции. Они потеряли деньги по сути.,1674490154
3,260957,"Уси-пуси, я обиделась...🤣",1673608933
4,260957,Есть же и в Евросоюзе вменяемые главы гос-в...,1673610014
...,...,...,...
190,244870,Ведь почти во всех регионах цены в самом деле ...,1630488189
191,244870,"Согласно статистике, в Алтае, Забайкалье, Калу...",1630488801
192,244870,"Рост цен на продукты неизбежен, но Путин старе...",1630489632
193,244870,Национальное достояние уходит не в те руки,1630508529


In [12]:
df.to_csv('expert_posts.csv', index=False)

In [13]:
data.to_csv('expert_comments.csv', index=False)